# Simple Example

Let's load our quickstart and translate to spanish

In [1]:
from pathlib import Path

In [2]:
config_folder = Path("configs")
md_file = Path("docs/quickstart.md")

In [3]:
from gpt_translate.translate import Translator

In [4]:
t = Translator(config_folder, language="es", max_chunk_tokens=400)

Model args: {'model': 'gpt-4', 'temperature': 0.7}


In [5]:
es_quickstart = t.translate_file(md_file)

Packing chunk 0 with 33 tokens
Packing chunk 1 with 17 tokens
Packing chunk 2 with 251 tokens
>> Translating 301 tokens
Packing chunk 3 with 190 tokens
Packing chunk 4 with 178 tokens
>> Translating 368 tokens
Packing chunk 5 with 380 tokens
>> Translating 380 tokens
Packing chunk 6 with 199 tokens
>> Translating 199 tokens
Packing chunk 7 with 244 tokens
>> Translating 244 tokens (last chunk)
Function translate_splitted_md took 195.9684 seconds to execute.
